In [1]:
# Import Libaries and Tools
import os
import sys
import glob
from scipy.io import readsav
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio

# Root directory of the project
ROOT_DIR = os.path.abspath("../../../../")

# Import ACWE Tools
sys.path.append(ROOT_DIR)
from ACWE_python_fall_2023 import acweSaveSeg_v5 as as5, acweRestoreScale as ars

In [ ]:
# Find Dataset

# Dataset
euvDataFolder = sorted(glob.glob(ROOT_DIR+'/Extensions/ISWAT/Dataset/Uncompressed/193/CHdataprocessedkm193*.save'))
hmiDataFolder = sorted(glob.glob(ROOT_DIR+'/Extensions/ISWAT/Dataset/Uncompressed/HMI/CHboundariesHMImagnetogramspart*.save'))

# SaveFolders - Update to refect location where data will be saved 
quackSaveFolder = '/mnt/coronal_holes/Paper 2/Code 02 Observations'
quackSaveFolder = quackSaveFolder + '/FinalPipeline/ISWAT/Standard/Results/' # alpha4/'
acweSaveFolder  = '/mnt/coronal_holes/Paper 1/CodeVI Observations/ISWAT/alpha3Sorted/' # alpha4Sorted/'

# Labels
labelFolder = ROOT_DIR+'/Extensions/ISWAT/Dataset/Coronal Hole Labels/Labels/'

# ACWE Prefix
quackPrefix = 'QUACK.' # Prefix for ACWE

In [ ]:
# Get Data

# Placeholder
euv_data = []
hmi_data = []

# Loop through Data
for i in range(len(euvDataFolder)):
    
    # Grab Files
    euvFile = euvDataFolder[i]
    hmiFile = hmiDataFolder[i]
    
    # Open Files
    euv_sav_data = readsav(euvFile)
    hmi_sav_data = readsav(hmiFile)
    euv_lst = list(euv_sav_data.keys())
    hmi_lst = list(hmi_sav_data.keys())
    
    # Unpack and Combine Data
    euv_data = [*euv_data,*euv_sav_data(euv_lst[0])]
    hmi_data = [*hmi_data,*hmi_sav_data(hmi_lst[0])]
    
acwe_data = sorted(glob.glob(acweSaveFolder + '*.npz'))

In [ ]:
# Cycle Through Dataset
for i in range(len(euv_data)):
    
    # EUV
    # Get EUV Entry
    euv_datum = euv_data[i]
    
    # Seperate EUV Image and Data
    ieuv = euv_datum[0]
    Heuv = []
    for j in range(1,len(euv_datum)):
        Heuv.append(euv_datum[j])
        
    # Convert into Pseudo Header - EUV
    euv_im_dims = np.array(ieuv.shape)
    c = (euv_im_dims/2) + 1
    euvSource = Heuv[5].decode('ascii')
    hEUV = {'RSUN':Heuv[14],
            'CDELT1':Heuv[2],
            'CDELT2':Heuv[3],
            'T_REC':Heuv[4].decode('ascii'),
            'CRPIX1':c[1],
            'CRPIX2':c[0]}
    
    # HMI
    # Get HMI Entry
    hmi_datum = hmi_data[i]
    
    # Seperate HMI Image and Data
    ihmi = hmi_datum[0]
    Hhmi = []
    for j in range(1,len(hmi_datum)):
        Hhmi.append(hmi_datum[j])
    
    # Convert into Pseudo Header - HMI Data
    hmi_im_dims = np.array(ihmi.shape)
    c = (hmi_im_dims/2) + 1
    hmiSource = Hhmi[5].decode('ascii')
    hHMI = {'RSUN':Hhmi[14],
            'CDELT1':Hhmi[2],
            'CDELT2':Hhmi[3],
            'T_REC':Hhmi[4].decode('ascii'),
            'CRPIX1':c[1],
            'CRPIX2':c[0]}
    
    #Get QUACK Data
    # Segmentation File Name
    quackFile = quackPrefix + euvSource + '.' + hEUV['T_REC'] \
               + '.' + hmiSource + '.' + hHMI['T_REC'] + '.npz'
    quackFile = quackFile.replace(' ','_').replace(':','')
    
    # Get Segmentation
    qH,qAH,qSEG = as5.openSeg(quackSaveFolder + quackFile)
    
    # Get ACWE Data
    aH,aAH,aSEG = as5.openSeg(acwe_data[i])
    
    
    # Get Label Data
    year = hEUV['T_REC'][7:11]
    day  = hEUV['T_REC'][0:2].replace(' ','0')
    time = hEUV['T_REC'][12:23].replace(':','_')
    #print(labelFolder + '*' + year + '*' + day + 'T' + time + '*annot.png')
    labelFile = glob.glob(labelFolder + '*' + year + '*' + day + 'T' + time + '*annot.png')[0]
    labelImg  = imageio.imread(labelFile)
    
    # Display Versions
    Ieuv = np.log10(np.clip(ieuv,20,5000))
    #Ihmi = np.clip(ihmi,-100,100)
    qSEG = ars.upscale(qSEG,qAH)
    aSEG = ars.upscale(aSEG,aAH)
    
    # Display All
    plt.figure(figsize=[15,15],dpi=200)
    title = hEUV['T_REC']
    plt.suptitle(title)
    plt.subplot(2,2,1)
    plt.imshow(np.flip(Ieuv,axis=0),cmap='gray')
    title = euvSource
    plt.title(title)
    #plt.axis(False)
    #plt.subplot(2,2,2)
    #plt.imshow(np.flip(Ihmi,axis=0),cmap='gray',vmin=-100,vmax=100)
    #title = hmiSource
    #plt.title(title)
    #plt.axis(False)
    plt.subplot(2,2,3)
    plt.imshow(np.flip(qSEG,axis=0),interpolation='None')
    title = quackPrefix.replace('.',' ') + 'Segmentation'
    plt.title(title)
    #plt.axis(False)
    plt.subplot(2,2,2)
    plt.imshow(labelImg)
    plt.axis(False)
    plt.subplot(2,2,4)
    plt.imshow(np.flip(aSEG,axis=0),interpolation='None')
    title = 'ACWE Segmentation'
    plt.title(title)
    #plt.axis(False)
    plt.show()
    
    print(qAH['ALPHA'])
    print("____")